In [1]:
#!pip install scrapy_proxies
!pip install scrapy-fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13489 sha256=ce4d127febb5e2ab87b4e7f13c1d9cec99ced67420e5d0585f5ba4ff154d3046
  Stored in directory: c:\users\diego\appdata\local\pip\cache\wheels\ae\e7\76\7dd44644d065268ab0e1b4fa2e802fa4bb0157717b7d6c6d92
Successfully built fake-useragent


In [1]:
import scrapy
import csv
import time
import scrapy_proxies
from scrapy.crawler import CrawlerProcess
import pandas as pd
#import scrapy-fake-useragent
class FirstWebsiteSpider(scrapy.Spider):
    
    rotate_user_agent = True   
    name = 'first_website'
    #https://www.amazon.com/s?k=data+science&i=stripbooks-intl-ship&page=3&crid=1YWU6OJK4L3BA&qid=1662895059&sprefix=data+science%2Cstripbooks-intl-ship%2C247&ref=sr_pg_3
    allowed_domains = ["www.amazon.com"] #, "https://www.amazon.com"
    handle_httpstatus_list = [503]
    start_urls =["https://www.amazon.com/s?k=python&ref=nb_sb_noss"] 
    list_items=[]
    j=1
    #download_delay = 3 #The number you write here will be how many seconds scrapy waits before sending another request.
    #x=pd.DataFrame([])
    n_page_to_scrape=7
    items_in_page=60
    RETRY_TIMES=20
    #DOWNLOADER_MIDDLEWARES = {
    #'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
    #'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
    #'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
    #'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
    #}
    FAKEUSERAGENT_PROVIDERS = [
    'scrapy_fake_useragent.providers.FakeUserAgentProvider',  # this is the first provider we'll try
    'scrapy_fake_useragent.providers.FakerProvider',  # if FakeUserAgentProvider fails, we'll use faker to generate a user-agent string for us
    'scrapy_fake_useragent.providers.FixedUserAgentProvider',  # fall back to USER_AGENT value
]
    def parse(self, response):
        print (response.request.headers['User-Agent'])
    def parse(self, response):
        #self.j=j
        print(self.j)
        #time.sleep(3)
        next_page_url=f"https://www.amazon.com/s?k=python&page={self.j}&qid=1663458276&ref=sr_pg_{self.j}"
        box=response.xpath("//div[@class='a-section a-spacing-small puis-padding-left-small puis-padding-right-small']")             #("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-3']")
        #print(box)
                                 
        #links=[]
        #titles=[]
        item={}
        
        for ind,i in enumerate(box):
            #print('g')
            #country_name = country.xpath(".//text()").get()
            #title=i.xpath("//span[@class='a-size-base a-color-base a-text-normal']/text()").getall()
            title=i.xpath(".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']/span/text()").get()
            link = i.xpath(".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']/@href").get()
            author=i.xpath(".//div[@class='a-row a-size-base a-color-secondary']/a[@class='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style']/text() | (.//div[@class='a-row a-size-base a-color-secondary']/a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style']/text()) | (.//div[@class='a-row a-size-base a-color-secondary']/span[@class='a-size-base']/text())").getall()
            
            #links.append(link)
            #'a-link-normal s-underline-text s-underline-link-text s-link-style'
            #titles.append(title)
            print(title)
            #//div[@class='a-section a-spacing-small puis-padding-left-small puis-padding-right-small']/*/*/a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']/@href  #print(title)
            yield response.follow(url=link, callback=self.parse_book, meta={'ind':ind,'title':title,'author':author,'link':link})
    
  
    # Getting data inside the "link" website
    def parse_book(self, response):
        #time.sleep(4)
        # Getting country names and each row element inside the population table
        title = response.request.meta['title']
        ind = response.request.meta['ind']
        link = response.request.meta['link']
        author = response.request.meta['author']
        #if author==[' ']:
        price=response.xpath("//div[@id='buyNew_cbb']/div[@class='a-row']/div[@class='a-column a-span12 a-text-left']/span/text() | ((//a[@class='a-link-normal']/span[@class='a-color-price']/text())[1]) | (//a[@class='a-button-text']/span[@class='a-color-base']/span[@class='a-size-base a-color-price a-color-price']/text())").get()
        #//div[@id='buyNew_cbb']/div[@class='a-row']/div[@class='a-column a-span12 a-text-left']/span/text() | ((//a[@class='a-link-normal']/span[@class='a-color-price']/text())[1]) 
        #pages=response.xpath("//span[@class='a-list-item']/*[contains(text(),'pages')/text()").get()
        pages=response.xpath("//span[@class='a-list-item']/span[contains( text(),'pages')]/text()").get()
        language=response.xpath("//span[@class='a-text-bold'][contains(text(),'Language')]/following-sibling::span[1]/text()").get()
        publisher=response.xpath("//span[@class='a-text-bold'][contains(text(),'Publisher')]/following-sibling::span[1]/text()").get()
        dimensions=response.xpath("//span[@class='a-text-bold'][contains(text(),'Dimensions')]/following-sibling::span[1]/text()").get()
        weight=response.xpath("//span[@class='a-text-bold'][contains(text(),'Weight')]/following-sibling::span[1]/text()").get()
        avg_reviews=response.xpath("(//a[@class='a-popover-trigger a-declarative']/i[@class='a-icon a-icon-star a-star-5']/span[@class='a-icon-alt']/text())[1] | ((//a[@class='a-popover-trigger a-declarative']/i/span[@class='a-icon-alt'])[1]/text())").get()
        time.sleep(0.1)
        star5=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[1]/text()").getall()
        star4=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[2]/text()").getall()
        star3=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[3]/text()").getall()
        star2=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[4]/text()").getall()
        star1=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[5]/text()").getall()
        ISBN_13=response.xpath("//div[@id='isbn_feature_div']/div/div/span[contains(text(),'13')]/following-sibling::span[1]/text()").get()
        n_reviews=response.xpath("(//span[@class='a-declarative']/a[@class='a-link-normal']/span[@id='acrCustomerReviewText']/text())[1]").get()
        
        #print(title)
        #author= response.xpath("//span[@class='a-declarative']/a[@class='a-link-normal contributorNameID']/text()").get()  # You can also use the whole class value  --> response.xpath('(//table[@class="table table-striped table-bordered table-hover table-condensed table-list"])[1]/tbody/tr')
            # Return data extracted
        keys=['title','author','price','pages','avg_reviews','n_reviews','star5','star4','star3','star2','star1','dimensions','weight','language','publisher','ISBN_13','link']
        values= [title,author,price,pages,avg_reviews,n_reviews,star5,star4,star3,star2,star1,dimensions,weight,language,publisher,ISBN_13,link]    #  {'title': title, 'author':author}
        self.list_items.append(values)
        item=dict(zip(keys, values))
        print(len(self.list_items))
        print(star5,avg_reviews,dimensions)
        #print(title,author)
        
      
        #if self.j >1:
          #  next_page_url=f"https://www.amazon.com/s?k=data+science&s=relevanceblender&page={self.j}&crid=2YKZLS0CJFDGI&qid=1663103507&sprefix=data+science%2Caps%2C828&ref=sr_pg_{self.j}"
        #else:
            #next_page_url=self.start_urls[0]
        #yield scrapy.Request(url=next_page_url, callback=self.parse_pagination, meta={'title':title,'author':author})
        
        #   def parse_pagination(self,response):
        #title = response.request.meta['title']  
        #author=response.request.meta['author'] 
        #print('va beeeeeneeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee')
        #if ind==
        #print(ind)
        
        if ind==59:
            time.sleep(3)
            self.j+=1
            next_page_url=f"https://www.amazon.com/s?k=python&page={self.j}&qid=1663458276&ref=sr_pg_{self.j}"
            if next_page_url and self.j<self.n_page_to_scrape:
                print('start_new_page')
            
                yield scrapy.Request(url=next_page_url, callback=self.parse)#,self.j 
            #time.sleep(10)
         #,'authors','dates'      
        if len(self.list_items)==(self.items_in_page): #self.j==self.n_page_to_scrape:
            time.sleep(5)
            print(self.list_items)
            x=pd.DataFrame(self.list_items,columns=keys)
            #print(x)
            yield x.to_csv("outputfile2_python.csv",sep=",")
            #with open("outputfile.csv", "w", newline="") as f:
                #writer = csv.DictWriter(f, ['titles','authors','dates'])
                #writer.writeheader()           
                #writer.writerows(self.list_items)
                #print(self.list_items) #
        if len(self.list_items)==359:#(self.n_page_to_scrape-2)*(self.items_in_page): #self.j==self.n_page_to_scrape:
            #time.sleep(70)
            #print(self.list_items)
            y=pd.DataFrame(self.list_items,columns=keys)
            #print(x)
            yield y.to_csv("outputfile_final_python.csv",sep=",")
            #with open("outputfile.csv", "w", newline="") as f:
                #writer = csv.DictWriter(f, ['titles','authors','dates'])
                #writer.writeheader()           
                #writer.writerows(self.list_items)
                #print(self.list_items) #
                
#//div[@class='a-row']/a[@class='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style']/text() | ( //div[@class='a-#row']/span[@class='a-size-base']/text()
process = CrawlerProcess({
     #'USER_AGENT':'USER_AGENT':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    #'USER_AGENT':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15"#
    'USER_AGENT':'Mozilla/5.0 (compatible; MSIE 7.0; Windows NT 5.1)'
  #'USER_AGENT':  "Mozilla/4.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36",
  #'USER_AGENT':  "Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36",
  #'USER_AGENT':  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.1.2 Safari/603.3.8",
  #'USER_AGENT':  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15"
  #'USER_AGENT':  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15"
  #'USER_AGENT':  "Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36"
  #'USER_AGENT':  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36"


})

process.crawl(FirstWebsiteSpider)
process.start()


2022-09-18 02:01:00 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-09-18 02:01:00 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.5 (default, May 18 2021, 14:42:02) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 3.4.7, Platform Windows-10-10.0.22000-SP0
2022-09-18 02:01:00 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2022-09-18 02:01:00 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-09-18 02:01:00 [scrapy.extensions.telnet] INFO: Telnet Password: 535d4442f6f3a24c
2022-09-18 02:01:00 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022-09-18 02:01:01 [scrapy.middleware] INFO: Enabled downloader middlewares:

1
Python Crash Course, 2nd Edition: A Hands-On, Project-Based Introduction to Programming
Python Programming for Beginners: The #1 Python Programming Crash Course for Beginners to Learn Python Coding Well & Fast (with Hands-On Exercises)
Python Distilled (Developer's Library)
Python Programming for Beginners: The Ultimate Crash Course to Learn Python in 7 Days with Step-by-Step Guidance and Hands-On Exercises
Python Crash Course, 2nd Edition: A Hands-On, Project-Based Introduction to Programming
Learning Python, 5th Edition
Automate the Boring Stuff with Python, 2nd Edition: Practical Programming for Total Beginners
Python Programming Language
Python Programming for Beginners: The #1 Python Programming Crash Course for Beginners to Learn Python Coding Well & Fast (with Hands-On Exercises)
Object-Oriented Python: Master OOP by Building Games and GUIs
Python Programming for Beginners: The Ultimate Crash Course to Learn Python in 7 Days with Step-by-Step Guidance and Hands-On Exercises
Py

2022-09-18 02:01:03 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (302) to <GET https://www.amazon.com/Python-Crash-Course-2nd-Edition/dp/1593279280/ref=sr_1_1_sspa?keywords=python&qid=1663459261&sr=8-1-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUExUjMwN0Q1SkRWMldTJmVuY3J5cHRlZElkPUEwOTIzMjc2M0FZRjcwTkpUMzg0RyZlbmNyeXB0ZWRBZElkPUEwODMxNTAzSFBFQjg1R1hRWkUwJndpZGdldE5hbWU9c3BfYXRmJmFjdGlvbj1jbGlja1JlZGlyZWN0JmRvTm90TG9nQ2xpY2s9dHJ1ZQ==> from <GET https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0831503HPEB85GXQZE0&url=%2FPython-Crash-Course-2nd-Edition%2Fdp%2F1593279280%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dpython%26qid%3D1663459261%26sr%3D8-1-spons%26psc%3D1&qualifier=1663459261&id=8731786596037934&widgetName=sp_atf>
2022-09-18 02:01:03 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (302) to <GET https://www.amazon.com/Python-Programming-Beginners-Step-Hands/dp/B0B3S3RFQT/ref=sr_1_4_sspa?keywords=python&qid=1663459261&

1
['\n            \n              97%\n            \n          '] 5.0 out of 5 stars None


2022-09-18 02:01:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Memorable-Python-Learn-Faster-ebook/dp/B07ZXJKKSN/ref=sr_1_21?keywords=python&qid=1663459261&sr=8-21> (referer: https://www.amazon.com/s?k=python&ref=nb_sb_noss)
2022-09-18 02:01:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Python-Beyond-Basics-Al-Sweigart/dp/1593279663/ref=sr_1_23?keywords=python&qid=1663459261&sr=8-23> (referer: https://www.amazon.com/s?k=python&ref=nb_sb_noss)
2022-09-18 02:01:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Python-2nd-Beginners-Hands-Project-ebook/dp/B071Z2Q6TQ/ref=sr_1_24?keywords=python&qid=1663459261&sr=8-24> (referer: https://www.amazon.com/s?k=python&ref=nb_sb_noss)


...

356
['\n            \n              76%\n            \n          '] 4.6 out of 5 stars 7.5 x 1.3 x 9.25 inches
357
['\n            \n              100%\n            \n          '] 5.0 out of 5 stars 9.1 x 1.2 x 6.9 inches


2022-09-18 02:03:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Automate-Boring-Stuff-Python-2nd/dp/1593279922/ref=sr_1_7?keywords=python&qid=1663459261&sr=8-7> (referer: https://www.amazon.com/s?k=python&ref=nb_sb_noss)


358
['\n            \n              82%\n            \n          '] 4.7 out of 5 stars 7 x 1.31 x 9.31 inches


2022-09-18 02:03:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Introducing-Python-Modern-Computing-Packages/dp/1492051365/ref=sr_1_13?keywords=python&qid=1663459261&sr=8-13> (referer: https://www.amazon.com/s?k=python&ref=nb_sb_noss)
2022-09-18 02:03:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Learning-Python-5th-Mark-Lutz/dp/1449355730/ref=sr_1_6?keywords=python&qid=1663459261&sr=8-6> (referer: https://www.amazon.com/s?k=python&ref=nb_sb_noss)
2022-09-18 02:03:28 [scrapy.core.engine] INFO: Closing spider (finished)
2022-09-18 02:03:28 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 370651,
 'downloader/request_count': 438,
 'downloader/request_method_count/GET': 438,
 'downloader/response_bytes': 81659466,
 'downloader/response_count': 438,
 'downloader/response_status_count/200': 366,
 'downloader/response_status_count/302': 72,
 'elapsed_time_seconds': 147.136148,
 'finish_reason': 'finished',


359
['\n            \n              81%\n            \n          '] 4.7 out of 5 stars 6.93 x 1.34 x 9.06 inches
360
['\n            \n              74%\n            \n          '] 4.5 out of 5 stars 7 x 2.8 x 9.19 inches


In [2]:
authors,titles=FirstWebsiteSpider.parse(scrapy.Spider,response)

NameError: name 'response' is not defined

In [ ]:
@href='/s?k=data+science&amp;i=stripbooks-intl-ship&amp;page=3&amp;crid=20YNDMUXOLCJ6&amp;qid=1663030048&amp;sprefix=data+science%2Cstripbooks-intl-ship%2C314&amp;ref=sr_pg_3' 

In [2]:
j=1
p=f'/s?k=data+science&amp;i=stripbooks-intl-ship&amp;page={j}&amp;crid=20YNDMUXOLCJ6&amp;qid=1663030048&amp;sprefix=data+science%2Cstripbooks-intl-ship%2C314&amp;ref=sr_pg_{j}'

In [3]:
p

'/s?k=data+science&amp;i=stripbooks-intl-ship&amp;page=1&amp;crid=20YNDMUXOLCJ6&amp;qid=1663030048&amp;sprefix=data+science%2Cstripbooks-intl-ship%2C314&amp;ref=sr_pg_1'

In [ ]:
<a href=>3</a>

In [3]:
u=0
a=f'rit{u}'
a

'rit0'

In [4]:
u=1
a

'rit0'

[3]

In [ ]:
#print(item)    
        self.j+=1
        print(self.j)
        next_page_url=f"https://www.amazon.com/s?k=data+science&s=relevanceblender&page={self.j}&crid=2YKZLS0CJFDGI&qid=1663103507&sprefix=data+science%2Caps%2C828&ref=sr_pg_{self.j}"
        if next_page_url and self.j<self.n_page_to_scrape:
            print('non va bene')
            yield response.follow(url=next_page_url, callback=self.parse)#,self.j  
         #,'authors','dates'
        if self.j==self.n_page_to_scrape:
            #print(self.list_items)
            x=pd.DataFrame(self.list_items,columns=['title','author'])
            #print(x)
            yield x.to_csv("outputfile2.csv",sep=",")
        #yield response.follow(url=next_page_url, callback=self.parse_pagination, meta=item )